In [1]:
#opens the each file and reads it and asks how many nucleotides to align
x = False
while x == False: 
    try: 
        #tries to open each of the files
        file1 = open(input ("File Name and Extension #1: "), "r")
        file2 = open(input ("File Name and Extension #2: "), "r")
        file3 = open(input ("File Name and Extension #3: "), "r")
        x = True
    except: 
        print("File Not Found. Enter Valid File Name! \n")

y = False
while y == False: 
    try: 
        #gets the first x nucleotides to align
        number = int(input("Number of Nucleotides to Align: "))
        y = True
    except:
        print("Enter a number!")

File Name and Extension #1: tor.txt
File Not Found. Enter Valid File Name! 

File Name and Extension #1: tor2.txt
File Name and Extension #2: wuhan.txt
File Name and Extension #3: urbani.txt
Number of Nucleotides to Align: 100


In [2]:
#makes the sequences into one line for easy readability
file1.readline()
seq1 = "\n".join(file1.readlines())[:number]
seq1 = seq1.replace("\n","")

file2.readline()
seq2 = "\n".join(file2.readlines())[:number]
seq2 = seq2.replace("\n","")

file3.readline()
seq3 = "\n".join(file3.readlines())[:number]
seq3 = seq3.replace("\n","")

#closes the files
file1.close()
file2.close()
file3.close()

In [3]:
from itertools import combinations 
#msa
sequences = [seq1, seq2, seq3]

#calcualtes the combinations of 2
seqs = combinations(sequences, 2) 

#variables to stores the outputs from the pairwise allignment
seq_info = []
identities = []

In [4]:
import numpy as np

#Dynamic Sequencing
def dynamic_seq(seq1, seq2):
    #creates the matrixes
    matrix = np.zeros((len(seq2)+1, len(seq1)+1))
    mutation = []
    for row in range(0, len(seq2)+1):
        values = []
        for col in range (0, len(seq1)+1):
            values.append([])
        mutation.append(values)



    #scores for Needleman gap, mismatch, match
    gap = -2
    match = 1
    mismatch = -1


    #first fill in the first row and column
    for row in range(1, len(seq2)+1):
        matrix[row][0] = matrix [row-1][0]+gap
        mutation [row][0].append("del")
    for col in range(1, len(seq1)+1):
        matrix[0][col] = matrix [0][col-1]+gap
        mutation [0][col].append("ins")

    #diagonal in the rest of the entries
    for row in range (1, len(seq2)+1):
        for col in range (1, len(seq1)+1):
            value1 = 0
            symbol = ""
            #calculates values for match/mismatch
            if seq1[col-1] == seq2[row-1]:
                value1 = matrix[row-1][col-1] + match
                symbol = "match"
            else:
                value1 = matrix[row-1][col-1] + mismatch
                symbol = "mismatch"
            #calculates values for gaps
            value2 = matrix [row-1][col]+gap
            value3 = matrix [row][col-1]+gap
            matrix[row][col] = max(value1, value2, value3)

            #assigns the symbol in the direction for where the value came from
            if matrix[row][col] == value1:
                mutation[row][col].append(symbol)
            elif matrix[row][col] == value2:
                mutation[row][col].append("del")
            elif matrix[row][col] == value3:
                mutation[row][col].append("ins")
    return mutation

In [5]:
#calculates the pairwise allignment 
def pairwise_allignment (seq1, seq2, mutation):
    align1 = ""
    align2 = ""
    final = ""
    i = len(seq2)
    j = len(seq1)
    
    #detemines the mutation and aligns the two sequences
    while i > 0 and j > 0:
        symbols = mutation[i][j]
        for symbol in symbols:
            if symbol == "mismatch":
                align1 += seq2 [i-1]
                align2 += seq1 [j-1]
                i -= 1
                j -= 1
                final += "*"
            elif symbol == "match":
                align1 += seq2 [i-1]
                align2 += seq1 [j-1]
                i -= 1
                j -= 1
                final += " "
            elif symbol == "del":
                align1 += seq2[i-1]
                align2 += "-"
                i -= 1
                final += "*"
            elif symbol == "ins":
                align1 += "-"
                align2 += seq1[j-1]
                j -= 1
                final += "*"
   
    #completes allignment in the ends
    while j > 0:
        align1 += "-"
        align2 += seq2[j-1]
        j -= 1
        final += "*"
    while i > 0:
        align1 += seq1[i-1]
        align2 += "-"
        i -= 1
        final += "*"

    #reorients the alignments
    align1 = align1 [::-1]
    align2 = align2 [::-1]
    final = final [::-1]
    
    #calcualtes the different percent_identities
    percent_identity = round(final.count (" ")/len(final)*100,2)
    return {"align1": align1,"align2": align2, "final": final, "percent_identity":percent_identity}

In [6]:
#once you get the combination, does the allignments and assigns the percent identities to identities
for seq in seqs: 
    mut = dynamic_seq(seq[0], seq[1])
    info = pairwise_allignment (seq[0], seq[1], mut)
    seq_info.append(info)
    identities.append(info.get("percent_identity"))

In [7]:
from statistics import median

#does the last two alignment
last_alignment = seq_info[identities.index(median(identities))]

#hard-coded ATT which is the first three nucleotides in Wuhan strain
##reason for having to start with the 1st nucleotide in this code; 
##why we did not do 3 different msa's each of 10,000 base pairs
#gets the alginment for the Tor2 strain and the Wuhan strain
if seq_info[identities.index(median(identities))].get("align1")[:3] == "ATT":
    tor2 = seq_info[identities.index(median(identities))].get("align2")
    wuhan = seq_info[identities.index(median(identities))].get("align1")
else: 
    tor2 = seq_info[identities.index(median(identities))].get("align1")
    wuhan = seq_info[identities.index(median(identities))].get("align2")

In [8]:
#determines which of the sequence is the Urbani sequence after doing the alignment
urbani_1 = seq_info[identities.index(max(identities))].get("align1")
urbani_2 = seq_info[identities.index(max(identities))].get("align2")

#does the two alignments with both the potential urbani strains
first_alignment_one = dynamic_seq(tor2, urbani_1)
first_align_info_one = pairwise_allignment(tor2, urbani_1, first_alignment_one)
first_alignment_two = dynamic_seq(tor2, urbani_2)
first_align_info_two = pairwise_allignment(tor2, urbani_2, first_alignment_two)

#determines highest percent identity of the two alignments
if first_align_info_one.get("percent_identity") > first_align_info_two.get("percent_identity"):
    first_align_info = first_align_info_two
else: 
    first_align_info = first_align_info_one 

#the urbani strain is determined
if first_align_info.get("align1") == tor2:
    urbani = first_align_info.get("align2")
else: 
    urbani = first_align_info.get("align1")

In [9]:
#calculates the final alignment
i = 0
final = ""
final1 = first_align_info.get("final")
final2 = last_alignment.get("final")
while i < len(final1):
    if final1[i] == "*" or final2[i] == "*":
        final = final + "*"
    else:
        final = final + " "
    i += 1

#calculates percent identity
percent_identity = round(final.count(" ")/len(final)*100,2)

In [10]:
#creates an output file that aligns the three sequences together
output = open("output.txt", "w")
i = 0

#determines reminder for the last line
reminder = len(final) % 50

#prints the allignments in sets of 50
while i < len(final)-reminder:
    output.write("Urbani: " + urbani[i:i+50]+" "+ str(i+50)+ "\n")
    output.write("Tor2:   " + tor2[i:i+50]+" "+ str(i+50)+ "\n")
    output.write("Wuhan:  " + wuhan[i:i+50]+" "+ str(i+50)+ "\n")
    output.write("Final:  " + final[i:i+50]+" "+ str(i+50)+ "\n \n")
    i = i + 50

#prints the last lines of the allignment
output.write("Urbani: " + urbani[i:i+reminder]+" "+ str(i+reminder)+ "\n")
output.write("Tor2:   " + tor2[i:i+reminder]+" "+ str(i+reminder)+ "\n")
output.write("Wuhan:  " + wuhan[i:i+reminder]+" "+ str(i+reminder)+ "\n")
output.write("Final:  " + final[i:i+reminder]+" "+ str(i+reminder)+ "\n \n")

#prints the percent identity
output.write("Percent Identity: " + str(percent_identity) + "%")
output.close()

In [ ]:
#opens the each file and reads it and asks how many nucleotides to align
x = False
while x == False: 
    try: 
        #tries to open each of the files
        file1 = open(input ("File Name and Extension #1: "), "r")
        file2 = open(input ("File Name and Extension #2: "), "r")
        file3 = open(input ("File Name and Extension #3: "), "r")
        x = True
    except: 
        print("File Not Found. Enter Valid File Name! \n")

y = False
while y == False: 
    try: 
        #gets the first x nucleotides to align
        number = int(input("Number of Nucleotides to Align: "))
        y = True
    except:
        print("Enter a number!")

#makes the sequences into one line for easy readability
file1.readline()
seq1 = "\n".join(file1.readlines())[:number]
seq1 = seq1.replace("\n","")

file2.readline()
seq2 = "\n".join(file2.readlines())[:number]
seq2 = seq2.replace("\n","")

file3.readline()
seq3 = "\n".join(file3.readlines())[:number]
seq3 = seq3.replace("\n","")

#closes the files
file1.close()
file2.close()
file3.close()

from itertools import combinations 
#msa
sequences = [seq1, seq2, seq3]

#calcualtes the combinations of 2
seqs = combinations(sequences, 2) 

#variables to stores the outputs from the pairwise allignment
seq_info = []
identities = []

import numpy as np

#Dynamic Sequencing
def dynamic_seq(seq1, seq2):
    #creates the matrixes
    matrix = np.zeros((len(seq2)+1, len(seq1)+1))
    mutation = []
    for row in range(0, len(seq2)+1):
        values = []
        for col in range (0, len(seq1)+1):
            values.append([])
        mutation.append(values)



    #scores for Needleman gap, mismatch, match
    gap = -2
    match = 1
    mismatch = -1


    #first fill in the first row and column
    for row in range(1, len(seq2)+1):
        matrix[row][0] = matrix [row-1][0]+gap
        mutation [row][0].append("del")
    for col in range(1, len(seq1)+1):
        matrix[0][col] = matrix [0][col-1]+gap
        mutation [0][col].append("ins")

    #diagonal in the rest of the entries
    for row in range (1, len(seq2)+1):
        for col in range (1, len(seq1)+1):
            value1 = 0
            symbol = ""
            #calculates values for match/mismatch
            if seq1[col-1] == seq2[row-1]:
                value1 = matrix[row-1][col-1] + match
                symbol = "match"
            else:
                value1 = matrix[row-1][col-1] + mismatch
                symbol = "mismatch"
            #calculates values for gaps
            value2 = matrix [row-1][col]+gap
            value3 = matrix [row][col-1]+gap
            matrix[row][col] = max(value1, value2, value3)

            #assigns the symbol in the direction for where the value came from
            if matrix[row][col] == value1:
                mutation[row][col].append(symbol)
            elif matrix[row][col] == value2:
                mutation[row][col].append("del")
            elif matrix[row][col] == value3:
                mutation[row][col].append("ins")
    return mutation

#calculates the pairwise allignment 
def pairwise_allignment (seq1, seq2, mutation):
    align1 = ""
    align2 = ""
    final = ""
    i = len(seq2)
    j = len(seq1)
    
    #detemines the mutation and aligns the two sequences
    while i > 0 and j > 0:
        symbols = mutation[i][j]
        for symbol in symbols:
            if symbol == "mismatch":
                align1 += seq2 [i-1]
                align2 += seq1 [j-1]
                i -= 1
                j -= 1
                final += "*"
            elif symbol == "match":
                align1 += seq2 [i-1]
                align2 += seq1 [j-1]
                i -= 1
                j -= 1
                final += " "
            elif symbol == "del":
                align1 += seq2[i-1]
                align2 += "-"
                i -= 1
                final += "*"
            elif symbol == "ins":
                align1 += "-"
                align2 += seq1[j-1]
                j -= 1
                final += "*"
   
    #completes allignment in the ends
    while j > 0:
        align1 += "-"
        align2 += seq2[j-1]
        j -= 1
        final += "*"
    while i > 0:
        align1 += seq1[i-1]
        align2 += "-"
        i -= 1
        final += "*"

    #reorients the alignments
    align1 = align1 [::-1]
    align2 = align2 [::-1]
    final = final [::-1]
    
    #calcualtes the different percent_identities
    percent_identity = round(final.count (" ")/len(final)*100,2)
    return {"align1": align1,"align2": align2, "final": final, "percent_identity":percent_identity}

#once you get the combination, does the allignments and assigns the percent identities to identities
for seq in seqs: 
    mut = dynamic_seq(seq[0], seq[1])
    info = pairwise_allignment (seq[0], seq[1], mut)
    seq_info.append(info)
    identities.append(info.get("percent_identity"))

from statistics import median

#does the last two alignment
last_alignment = seq_info[identities.index(median(identities))]

#hard-coded ATT which is the first three nucleotides in Wuhan strain
##reason for having to start with the 1st nucleotide in this code; 
##why we did not do 3 different msa's each of 10,000 base pairs
#gets the alginment for the Tor2 strain and the Wuhan strain
if seq_info[identities.index(median(identities))].get("align1")[:3] == "ATT":
    tor2 = seq_info[identities.index(median(identities))].get("align2")
    wuhan = seq_info[identities.index(median(identities))].get("align1")
else: 
    tor2 = seq_info[identities.index(median(identities))].get("align1")
    wuhan = seq_info[identities.index(median(identities))].get("align2")

#determines which of the sequence is the Urbani sequence after doing the alignment
urbani_1 = seq_info[identities.index(max(identities))].get("align1")
urbani_2 = seq_info[identities.index(max(identities))].get("align2")

#does the two alignments with both the potential urbani strains
first_alignment_one = dynamic_seq(tor2, urbani_1)
first_align_info_one = pairwise_allignment(tor2, urbani_1, first_alignment_one)
first_alignment_two = dynamic_seq(tor2, urbani_2)
first_align_info_two = pairwise_allignment(tor2, urbani_2, first_alignment_two)

#determines highest percent identity of the two alignments
if first_align_info_one.get("percent_identity") > first_align_info_two.get("percent_identity"):
    first_align_info = first_align_info_two
else: 
    first_align_info = first_align_info_one 

#the urbani strain is determined
if first_align_info.get("align1") == tor2:
    urbani = first_align_info.get("align2")
else: 
    urbani = first_align_info.get("align1")

#calculates the final alignment
i = 0
final = ""
final1 = first_align_info.get("final")
final2 = last_alignment.get("final")
while i < len(final1):
    if final1[i] == "*" or final2[i] == "*":
        final = final + "*"
    else:
        final = final + " "
    i += 1

#calculates percent identity
percent_identity = round(final.count(" ")/len(final)*100,2)

#creates an output file that aligns the three sequences together
output = open("output.txt", "w")
i = 0

#determines reminder for the last line
reminder = len(final) % 50

#prints the allignments in sets of 50
while i < len(final)-reminder:
    output.write("Urbani: " + urbani[i:i+50]+" "+ str(i+50)+ "\n")
    output.write("Tor2:   " + tor2[i:i+50]+" "+ str(i+50)+ "\n")
    output.write("Wuhan:  " + wuhan[i:i+50]+" "+ str(i+50)+ "\n")
    output.write("Final:  " + final[i:i+50]+" "+ str(i+50)+ "\n \n")
    i = i + 50

#prints the last lines of the allignment
output.write("Urbani: " + urbani[i:i+reminder]+" "+ str(i+reminder)+ "\n")
output.write("Tor2:   " + tor2[i:i+reminder]+" "+ str(i+reminder)+ "\n")
output.write("Wuhan:  " + wuhan[i:i+reminder]+" "+ str(i+reminder)+ "\n")
output.write("Final:  " + final[i:i+reminder]+" "+ str(i+reminder)+ "\n \n")

#prints the percent identity
output.write("Percent Identity: " + str(percent_identity) + "%")
output.close()